In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [14]:
from generate_video import *

In [15]:
dset = gather_videos()

WARN: Missing bbox /beegfs/ua349/lstm/Penn_Action/labels/1154

WARN: Missing bbox /beegfs/ua349/lstm/Penn_Action/labels/1865



In [16]:
# import os, sys
# for ii in range(100 * 100):
#     sys.stdout.write('%d/%d\r' % (ii, 3000))
#     sys.stdout.flush()
videos, masks, targets = next_video_batch(dset, bsize=1)

In [17]:
print(videos.shape)
print(masks.shape)
print(targets.shape)

vid1 = videos[0]
heat1 = targets[0]
mask1 = masks[0]

(1, 4, 368, 368, 3)
(1, 4, 46, 46, 19)
(1, 4, 46, 46, 19)


In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(14, 10))
plt.imshow(mask1[:, :, 0])
plt.show()

In [ ]:

import matplotlib.pyplot as plt

heat1 = targets[0]

plt.figure(figsize=(14, 10))
plt.imshow(heat1[0][:,:, -1])
plt.show()

In [ ]:

import matplotlib.pyplot as plt




plt.figure(figsize=(14, 10))
for ii in range(4):
    plt.subplot(3, 4, ii+1)
    plt.imshow(vid1[ii]/256)
for ii in range(4):
    plt.subplot(3, 4, 4+ii+1)
    img = np.sum(heat1[ii][:-1], axis=-1)
    plt.imshow(img)
for ii in range(4):
    plt.subplot(3, 4, 8+ii+1)
    img = mask1[ii]
    plt.imshow(np.sum(img, axis=-1))

plt.show()